In [1]:
import os
os.chdir("../")

In [2]:
from pdf_parser import pdf_to_text
import openai

In [3]:
file_path = "PDFs/operating_systems.pdf"
docs = pdf_to_text(file_path,chunk_size=2000)

In [4]:
prompt_template = """Content: {}.\nBased on this content, create a {} multiple choice question and answer pair with 4 options and single correct answer to test the knowledge of the user in an exam. Response should be in the follwoing template:\nQuestion: <question> \nA) <option A> \nB) <option B> \n<option C> \n<option D> \nAnswer: <correct option>"""

In [8]:
docs[0].page_content

'**UNIT 1** ****\n**AN INTRODUCTION TO OPERATING SYSTEMS** ****\n\n**Application software** performs specific task for the user. \n**System software** operates and controls the computer system and provides a platform to run\napplication software.\n\nAn **operating system** is a piece of software that manages all the resources of a computer\nsystem, both hardware and software, and provides an environment in which the user can\nexecute his/her programs in a convenient and efficient manner by hiding underlying\ncomplexity of the hardware and acting as a resource manager.\n\nWhy OS?\n1. What if there is no OS?\na. Bulky and complex app. (Hardware interaction code must be in app’s\ncode base)\nb. Resource exploitation by 1 App.\nc. No memory protection.\n2. What is an OS made up of?\na. Collection of system software.\n\nAn operating system function - \n- Access to the computer hardware. \n- interface between the user and the computer hardware\n- **Resource management (Aka, Arbitration) (mem

In [5]:
api_key = "esecret_35mu1unb6nmdmy5vpyc5jdey8c"
client = openai.OpenAI(
    base_url = "https://api.endpoints.anyscale.com/v1",
    api_key = "esecret_35mu1unb6nmdmy5vpyc5jdey8c"
)
prompt = prompt_template.format(docs[4].page_content,"set of 3")
chat_completion = client.chat.completions.create(
    model="meta-llama/Llama-2-13b-chat-hf",
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": prompt}],
    temperature=0.5
)
resp_text = chat_completion.model_dump()['choices'][0]['message']['content']

In [6]:
print(resp_text)

  Sure, here are three multiple choice questions based on the content you provided:

Question 1: What is the main difference between thread context switching and process context switching?

A) Thread context switching does not include switching of memory address space, while process context switching does.
B) Thread context switching includes switching of memory address space, while process context switching does not.
C) Both thread and process context switching include switching of memory address space.
D) Thread context switching is faster than process context switching.

Answer: A) Thread context switching does not include switching of memory address space, while process context switching does.

Question 2: What is the function of the kernel in an operating system?

A) To provide a user interface for users to interact with the operating system.
B) To manage and control I/O operations and I/O devices.
C) To allocate and deallocate memory space as per the need of the system.
D) To sch

In [7]:
def convert_resp_text_to_mcqs(resp_text):
    resp_text_lines = resp_text.split('\n')
    lb = "<line_break>"
    resp_text_lines = [line+lb for line in resp_text_lines]
    q_lst, ans_lst = [], []

    q = -1
    question, answer = "", ""
    for line in resp_text_lines:
        if line.strip().replace(lb,"")=="":
            continue
        if line.lower().strip().startswith('question'):
            if answer.strip() != "":
                ans_lst.append(answer.strip().replace(lb,"\n"))
            question = line.split(":")[1].strip()
            q = 1
        elif line.lower().strip().startswith('answer'):
            if question.strip() != "":
                q_lst.append(question.strip().replace(lb,"\n"))
            q = 0
            answer = line.split(":")[1].strip()
        elif q==1:
            question += " "
            question += line.strip()
        elif q==0:
            answer += " "
            answer += line.strip()
    if answer.strip()!="":
        ans_lst.append(answer.strip().replace(lb,"\n"))
    return q_lst, ans_lst

In [8]:
q_lst, ans_lst = convert_resp_text_to_mcqs(resp_text)

In [19]:
def get_options(qa):
    q = qa['question']
    a = qa['answer']
    f = 1
    q_lines = q.split("\n")
    options = {}
    question = ""
    for line in q_lines:
        if line.strip().startswith('A)')==False and f==1:
            question += line.strip() + "\n"
        if line.strip().startswith('A)'):
            f = 0
            options['A'] = line.strip()[2:].strip()
        if line.strip().startswith('B)'):
            options['B'] = line.strip()[2:].strip()
        if line.strip().startswith('C)'):
            options['C'] = line.strip()[2:].strip()
        if line.strip().startswith('D)'):
            options['D'] = line.strip()[2:].strip()

    if "A)" in a:
        correct_option = 'A'
    elif "B)" in a:
        correct_option = "B"
    elif "C)" in a:
        correct_option = "C"
    else:
        correct_option = "D"
    mcq = {}
    mcq['question'] = question.strip()
    mcq['answer'] = correct_option
    mcq['A'],mcq['B'],mcq['C'],mcq['D'] = options['A'],options['B'],options['C'],options['D']
    return mcq

In [20]:
get_options({"question":q_lst[0],"answer":ans_lst[0]})

{'question': 'What is the main difference between thread context switching and process context switching?',
 'answer': 'A',
 'A': 'Thread context switching does not include switching of memory address space, while process context switching does.',
 'B': 'Thread context switching includes switching of memory address space, while process context switching does not.',
 'C': 'Both thread and process context switching include switching of memory address space.',
 'D': 'Thread context switching is faster than process context switching.'}